## Face extraction using visual words

---

In this notebook I will use bag-of-visual generated by the [last notebook](https://colab.research.google.com/drive/1J5B1rTAGaAfFelf8P9d4lXzjjH1j_WBr?usp=sharing) and apply multiple models in order to select the best for bounding box. 

---

references:
  - sklearn for multiregression : [models](https://scikit-learn.org/stable/modules/multiclass.html)
  - save and load models : [post](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [1]:
!gdown 1ptO1oDhO4FLj0GF41yA0FSnM3rx-Mpae

Downloading...
From: https://drive.google.com/uc?id=1ptO1oDhO4FLj0GF41yA0FSnM3rx-Mpae
To: /content/final_sift_data.pkl
100% 286M/286M [00:01<00:00, 145MB/s]


In [2]:
!mkdir models

In [3]:
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC 
from sklearn import metrics
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import ast


tqdm.pandas()
sns.set_theme()
MODEL_DIR_NAME = 'models/'

In [4]:
def validation_metrics(y_pred, y_true, show_img=True):
  rmse = np.sqrt(np.square(y_pred - y_true).mean(axis=-1)) * 200
  fig = px.histogram(rmse)
  fig.show()
  return np.mean(rmse)

In [5]:
df = pd.read_pickle('final_sift_data.pkl')
# df['visual_words_histogram'] = [np.array(ast.literal_eval(cur_vword)) for cur_vword in df['visual_words_histogram']]
df[['x0', 'y0', 'x1', 'y1']] = df[['x0', 'y0', 'x1', 'y1']] / 200 
df['sift_p_arr_hist'] = df['sift_p_arr_hist'].apply(lambda x: x/sum(x))
# df['sift_p_arr_f_p'] = df['sift_p_arr_f_p'].apply(lambda x: x/sum(x))
# df['sift_p_arr_f_p_q'] = df['sift_p_arr_f_p_q'].apply(lambda x: x/sum(x))

df.head(3)

,img_location,x0,y0,x1,y1,sift_p_arr_f_p,sift_p_arr_f_p_q,sift_p_arr_hist
0,celeb_data_resized/000001.jpg,0.232274,0.103348,0.784841,0.558952,"[1137, 1691, 1730, 1730, 1880, 2071, 2062, 230...","[14, 22, 22, 22, 24, 27, 27, 30, 32, 35, 37, 4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,celeb_data_resized/000002.jpg,0.170213,0.158249,0.692671,0.673401,"[552, 964, 964, 1143, 1356, 2169, 2169, 2149, ...","[7, 12, 12, 14, 17, 28, 28, 27, 27, 30, 30, 35...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0032467532467..."
2,celeb_data_resized/000003.jpg,0.432000,0.209964,0.614000,0.658363,"[2451, 3497, 3787, 4119, 4119, 4285, 4922, 488...","[34, 49, 53, 57, 57, 60, 68, 68, 77, 93, 96, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [7]:
len(df_train), len(df_test)

(7028, 3012)

In [8]:
x_train = np.array(list(df_train['sift_p_arr_hist'].values))
y_train = np.array(list(df_train[['x0', 'y0', 'x1', 'y1']].values))

x_test = np.array(list(df_test['sift_p_arr_hist'].values))
y_test = np.array(list(df_test[['x0', 'y0', 'x1', 'y1']].values))

In [9]:
def train_model_and_test_sk_model(x_train, y_train, x_test_, y_test, sk_model, filename):
  sk_model.fit(x_train, y_train)
  y_pred = sk_model.predict(x_test) 
  pickle.dump(sk_model, open(MODEL_DIR_NAME + filename, 'wb'))

  print("rsme : ", validation_metrics(y_test, y_pred))


## Defining the models for bounding box generation

---

Now we define a model predict the bounding boxes of the image based on bag of visual words. 

So first we can try some base models :

 - GradientBoostingRegressor
 - SGDRegressor
 - KNNr
 - simple NN


In order to check the model performance we generate the histogram of RMSE of each example, them we display the mean of the RMSE distribution. The perfect model will have a mean RMSE equal to zero and a histogram concentrated around zero.

#### SKlearn gradient boosting

---

In [10]:
sk_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test,
                              sk_model, filename='gradient_model.sav')

      Iter       Train Loss   Remaining Time 
         1           0.0140           40.85s
         2           0.0139           39.98s
         3           0.0138           39.76s
         4           0.0137           39.19s
         5           0.0136           38.71s
         6           0.0135           38.09s
         7           0.0134           37.44s
         8           0.0133           36.96s
         9           0.0132           36.47s
        10           0.0131           35.98s
        20           0.0124           31.95s
        30           0.0118           27.88s
        40           0.0113           23.76s
        50           0.0108           19.81s
        60           0.0104           15.85s
        70           0.0101           11.85s
        80           0.0098            7.89s
        90           0.0095            3.95s
       100           0.0093            0.00s
      Iter       Train Loss   Remaining Time 
         1           0.0045           40.70s
        

rsme :  23.79108154469082


### SVG for regression

---

In [11]:
from sklearn.linear_model import SGDRegressor

In [12]:
sk_model = MultiOutputRegressor(SGDRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'SGDRegressor.sav')

-- Epoch 1
Norm: 0.01, NNZs: 508, Bias: 0.284642, T: 7028, Avg. loss: 0.007943
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 0.02, NNZs: 508, Bias: 0.289667, T: 14056, Avg. loss: 0.007071
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 0.02, NNZs: 508, Bias: 0.283765, T: 21084, Avg. loss: 0.007074
Total training time: 0.05 seconds.
-- Epoch 4
Norm: 0.02, NNZs: 508, Bias: 0.283230, T: 28112, Avg. loss: 0.007071
Total training time: 0.06 seconds.
-- Epoch 5
Norm: 0.02, NNZs: 508, Bias: 0.282732, T: 35140, Avg. loss: 0.007069
Total training time: 0.08 seconds.
-- Epoch 6
Norm: 0.02, NNZs: 508, Bias: 0.286608, T: 42168, Avg. loss: 0.007067
Total training time: 0.09 seconds.
Convergence after 6 epochs took 0.09 seconds
-- Epoch 1
Norm: 0.01, NNZs: 508, Bias: 0.146485, T: 7028, Avg. loss: 0.002482
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 0.01, NNZs: 508, Bias: 0.144275, T: 14056, Avg. loss: 0.002256
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 0.01, NNZs: 508, Bia

rsme :  24.765199347207624


KNN for regression

--- 

In [13]:
from sklearn.neighbors import KNeighborsRegressor

In [14]:
sk_model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=10))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'knnRegression.sav')

rsme :  29.933526881268726


Simple NN for regression

--- 

In [15]:
from sklearn.neural_network import MLPRegressor

In [16]:
sk_model = MultiOutputRegressor(MLPRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'simpleNN.sav')

Iteration 1, loss = 0.01490191
Iteration 2, loss = 0.00698018
Iteration 3, loss = 0.00669752
Iteration 4, loss = 0.00648650
Iteration 5, loss = 0.00633229
Iteration 6, loss = 0.00616684
Iteration 7, loss = 0.00604820
Iteration 8, loss = 0.00597840
Iteration 9, loss = 0.00590692
Iteration 10, loss = 0.00585401
Iteration 11, loss = 0.00581601
Iteration 12, loss = 0.00577979
Iteration 13, loss = 0.00575875
Iteration 14, loss = 0.00573585
Iteration 15, loss = 0.00571559
Iteration 16, loss = 0.00570519
Iteration 17, loss = 0.00568150
Iteration 18, loss = 0.00565656
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.00362235
Iteration 2, loss = 0.00231029
Iteration 3, loss = 0.00227685
Iteration 4, loss = 0.00225421
Iteration 5, loss = 0.00223768
Iteration 6, loss = 0.00222201
Iteration 7, loss = 0.00220846
Iteration 8, loss = 0.00219305
Iteration 9, loss = 0.00218812
Iteration 10, loss = 0.00217083
Iteration 11, loss = 0.00216664


rsme :  24.088740118131962


In [17]:
!zip -r models_for_face_extraction_celeb.zip models

  adding: models/ (stored 0%)
  adding: models/SGDRegressor.sav (deflated 9%)
  adding: models/gradient_model.sav (deflated 63%)
  adding: models/simpleNN.sav (deflated 3%)
  adding: models/knnRegression.sav (deflated 94%)
